In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [28]:
# data 설정하기
data = pd.read_csv('../dataset/heartattack/heart.csv')

print(data.head(50))
print("\n")
print(data.dtypes)

data['sex'] = data['sex'].astype('category') # 성별 category
data['cp'] =  data['cp'].astype('category') # 가슴통증 category
data['fbs'] = data['fbs'].astype('category') # 혈중 당 category
data['restecg'] = data['restecg'].astype('category')
data['exng'] = data['exng'].astype('category')
data['output'] = data['output'].astype('category')

    age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0    63    1   3     145   233    1        0       150     0      2.3    0   
1    37    1   2     130   250    0        1       187     0      3.5    0   
2    41    0   1     130   204    0        0       172     0      1.4    2   
3    56    1   1     120   236    0        1       178     0      0.8    2   
4    57    0   0     120   354    0        1       163     1      0.6    2   
5    57    1   0     140   192    0        1       148     0      0.4    1   
6    56    0   1     140   294    0        0       153     0      1.3    1   
7    44    1   1     120   263    0        1       173     0      0.0    2   
8    52    1   2     172   199    1        1       162     0      0.5    2   
9    57    1   2     150   168    0        1       174     0      1.6    2   
10   54    1   0     140   239    0        1       160     0      1.2    2   
11   48    0   2     130   275    0        1       139     0    

In [37]:
# 전처리
trainX, testX, trainY, testY = \
    train_test_split(data[[col for col in data.columns if col not in 'output']], data['output'], test_size=0.2)

knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(trainX, trainY)

KNeighborsClassifier()

In [38]:
predY = knn.predict(testX)
print((predY == testY).mean())

0.7213114754098361
